### 請使用 ImageDataGenerator 來進行 Cifar-10 資料集的訓練，並觀察不同的圖像增強方法是否會顯著影響訓練結果，作業請直接提交Day099_data_augmentation.ipynb

> 經過實驗發現 (1) 的 Test accuracy: 0.72 比 (2) 的 Test accuracy: 0.66 還要好
> (1) ImageDataGenerator(rotation_range=15, width_shift_range=0.1,height_shift_range=0.1,
                                   shear_range=0.1, zoom_range=0.1,
                                   horizontal_flip=True)
> (2) ImageDataGenerator(rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [ ]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [ ]:
# 此函數會幫我們把多張影像畫成一張多宮格圖
def img_combine(img, ncols=8, size=1, path=False):
    from math import ceil
    import matplotlib.pyplot as plt
    import numpy as np
    nimg = len(img)
    nrows = int(ceil(nimg/ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, sharex=True, sharey=True, figsize=(ncols*size,nrows*size))
    if nrows == 0:
        return
    elif ncols == 1:
        for r, ax in zip(np.arange(nrows), axes):
            nth=r
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                
            ax.set_axis_off()
    elif nrows == 1:
        for c, ax in zip(np.arange(ncols), axes):
            nth=c
            if nth < nimg:
                ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
            ax.set_axis_off()
    else:
        for r, row in zip(np.arange(nrows), axes):
            for c, ax in zip(np.arange(ncols), row):
                nth=r*ncols+c
                if nth < nimg:
                    ax.imshow(img[nth], cmap='rainbow', vmin=0, vmax=1)
                ax.set_axis_off()
    plt.show()

In [ ]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f'x_train.shape: {x_train.shape}')

In [ ]:
# 取前 32 張圖片做視覺化
images = x_train[:32]
img_combine(images)

In [ ]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
# 注意！！ ImageDataGenerator 是一個 Generator (生成器)! 對 Generator 不熟悉的同學請回到 Day098 做複習。
# 使用 .flow 後，就會對我們的影像進行增強，再 call next 取出 generator 的圖像。(shuffle=False 因為我們希望圖像的順序不要改變，方便觀察。實際訓練時預設是 shuffle=True) 
augmented_iamges = next(data_generator.flow(images, shuffle=False))
img_combine(augmented_iamges.astype("int")) # 注意在訓練時神經網路時，圖像資料必須要是 float32，但在做視覺化時要轉為 int 才能順利畫圖。所以為了畫圖才把資料轉為 int

In [ ]:
# 因為隨機性的關係，所以一樣的圖像再經過一次 generator 後的結果不一定相同
augmented_iamges = next(data_generator.flow(images, shuffle=False))
img_combine(augmented_iamges.astype("int"))

## 作業

請使用 ImageDataGenerator 來進行 Cifar-10 資料集的訓練，並觀察不同的圖像增強方法是否會顯著影響訓練結果

In [1]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

2021-12-03 15:24:00.423736: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-03 15:24:00.423775: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f'x_train.shape: {x_train.shape}')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train.shape: (50000, 32, 32, 3)


In [3]:
model = Sequential()

# chanel*(kernel)*filters+bias: chanel=3, kernel=(3,3), filters=32, bias=32
print(3*(3*3)*32+32) 
model.add(Conv2D(32, (3,3), padding='same', 
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))

# previous outputs * (kernel) * filters + bias: previous outputs=32, kernel=(3,3), filters=32, bias=32
print(32*(3*3)*32+32) 
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

# previous outputs * (kernel) * filters + bias: previous outputs=32, kernel=(3,3), filters=64, bias=64
print(32*(3*3)*64+64)
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))

# previous outputs * (kernel) * filters + bias: previous outputs=64, kernel=(3,3), filters=64, bias=64
print(64*(3*3)*64+64)
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

print(f'flatter: {5*5*64}')
model.add(Flatten())

print(f'dense_1: {1600*512+512}')
model.add(Dense(512))
model.add(Activation('relu'))

model.add(Dropout(0.5))

# previous outputs * dense neurons + bias
print(f'dense_4: {512*10+10}')
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()     

model.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

896
9248
18496
36928
flatter: 1600
dense_1: 819712
dense_4: 5130
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________

2021-12-03 15:24:08.320017: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-03 15:24:08.320073: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-03 15:24:08.320111: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (11402.pyspark3): /proc/driver/nvidia/version does not exist
2021-12-03 15:24:08.320434: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
print(f'x_train.shape: {x_train.shape}')
print(f'x_test.shape: {x_test.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'y_test.shape: {y_test.shape}')

x_train.shape: (50000, 32, 32, 3)
x_test.shape: (10000, 32, 32, 3)
y_train.shape: (50000, 10)
y_test.shape: (10000, 10)


In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


train_datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
         validation_data=(x_test, y_test),
            epochs=epochs)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


/home/wanzi/ml/cupoyMarathon/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-12-03 15:24:15.617876: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
391/391 [==============================] - 66s 168ms/step - loss: 1.9487 - accuracy: 0.2828 - val_loss: 1.5315 - val_accuracy: 0.4518
Epoch 2/10
391/391 [==============================] - 65s 167ms/step - loss: 1.6977 - accuracy: 0.3864 - val_loss: 1.4207 - val_accuracy: 0.4961
Epoch 3/10
391/391 [==============================] - 65s 166ms/step - loss: 1.5705 - accuracy: 0.4328 - val_loss: 1.5200 - val_accuracy: 0.4665
Epoch 4/10
391/391 [==============================] - 65s 167ms/step - loss: 1.4820 - accuracy: 0.4702 - val_loss: 1.3175 - val_accuracy: 0.5439
Epoch 5/10
391/391 [==============================] - 63s 161ms/step - loss: 1.4125 - accuracy: 0.4943 - val_loss: 1.2037 - val_accuracy: 0.5757
Epoch 6/10
391/391 [==============================] - 66s 169ms/step - loss: 1.3627 - accuracy: 0.5148 - val_loss: 1.1835 - val_accuracy: 0.5859
Epoch 7/10
391/391 [==============================] - 68s 173ms/step - loss: 1.3263 - accuracy: 0.5314 - val_loss: 1.0997 - val_ac

In [7]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
         validation_data=(x_test, y_test),
            epochs=epochs)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1/10
391/391 [==============================] - 65s 167ms/step - loss: 1.6224 - accuracy: 0.4298 - val_loss: 1.3406 - val_accuracy: 0.5443
Epoch 2/10
391/391 [==============================] - 64s 163ms/step - loss: 1.4496 - accuracy: 0.4955 - val_loss: 1.4717 - val_accuracy: 0.5200
Epoch 3/10
391/391 [==============================] - 65s 166ms/step - loss: 1.3739 - accuracy: 0.5245 - val_loss: 1.2541 - val_accuracy: 0.5635
Epoch 4/10
391/391 [==============================] - 65s 165ms/step - loss: 1.3412 - accuracy: 0.5364 - val_loss: 1.0494 - val_accuracy: 0.6278
Epoch 5/10
391/391 [==============================] - 65s 166ms/step - loss: 1.3074 - accuracy: 0.5467 - val_loss: 1.0866 - val_accuracy: 0.6488
Epoch 6/10
391/391 [==============================] - 65s 166ms/step - loss: 1.2887 - accuracy: 0.5557 - val_loss: 1.1378 - val_accuracy: 0.6280
Epoch 7/10
391/391 [==============================] - 68s 173ms/step - loss: 1.2648 - accuracy: 0.5640 - val_loss: 1.1174 - val_ac

In [8]:
train_datagen = ImageDataGenerator(rotation_range=90,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
         validation_data=(x_test, y_test),
            epochs=epochs)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1/10
391/391 [==============================] - 68s 174ms/step - loss: 1.4616 - accuracy: 0.4919 - val_loss: 1.1141 - val_accuracy: 0.6171
Epoch 2/10
391/391 [==============================] - 68s 173ms/step - loss: 1.4328 - accuracy: 0.5026 - val_loss: 1.1414 - val_accuracy: 0.6107
Epoch 3/10
391/391 [==============================] - 68s 175ms/step - loss: 1.4210 - accuracy: 0.5059 - val_loss: 1.1349 - val_accuracy: 0.6009
Epoch 4/10
391/391 [==============================] - 68s 175ms/step - loss: 1.4118 - accuracy: 0.5078 - val_loss: 1.4061 - val_accuracy: 0.5646
Epoch 5/10
391/391 [==============================] - 67s 172ms/step - loss: 1.4041 - accuracy: 0.5134 - val_loss: 1.4052 - val_accuracy: 0.5439
Epoch 6/10
391/391 [==============================] - 67s 171ms/step - loss: 1.3957 - accuracy: 0.5163 - val_loss: 1.0947 - val_accuracy: 0.6178
Epoch 7/10
391/391 [==============================] - 67s 171ms/step - loss: 1.3839 - accuracy: 0.5216 - val_loss: 1.2524 - val_ac

In [9]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )

train_datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
         validation_data=(x_test, y_test),
            epochs=epochs)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Epoch 1/10
391/391 [==============================] - 67s 172ms/step - loss: 1.0788 - accuracy: 0.6339 - val_loss: 1.7640 - val_accuracy: 0.5150
Epoch 2/10
391/391 [==============================] - 67s 172ms/step - loss: 1.0282 - accuracy: 0.6552 - val_loss: 0.8523 - val_accuracy: 0.7058
Epoch 3/10
391/391 [==============================] - 67s 172ms/step - loss: 0.9968 - accuracy: 0.6647 - val_loss: 0.9993 - val_accuracy: 0.6789
Epoch 4/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9833 - accuracy: 0.6666 - val_loss: 0.8503 - val_accuracy: 0.7112
Epoch 5/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9845 - accuracy: 0.6703 - val_loss: 0.8434 - val_accuracy: 0.7091
Epoch 6/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9735 - accuracy: 0.6743 - val_loss: 0.8715 - val_accuracy: 0.7063
Epoch 7/10
391/391 [==============================] - 67s 172ms/step - loss: 0.9711 - accuracy: 0.6760 - val_loss: 0.8831 - val_ac

In [10]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

train_datagen.fit(x_train)
# fits the model on batches with real-time data augmentation:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
         validation_data=(x_test, y_test),
            epochs=epochs)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9695 - accuracy: 0.6771 - val_loss: 0.8320 - val_accuracy: 0.7280
Epoch 2/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9656 - accuracy: 0.6793 - val_loss: 0.8588 - val_accuracy: 0.7072
Epoch 3/10
391/391 [==============================] - 69s 176ms/step - loss: 0.9659 - accuracy: 0.6795 - val_loss: 0.8070 - val_accuracy: 0.7336
Epoch 4/10
391/391 [==============================] - 68s 175ms/step - loss: 0.9651 - accuracy: 0.6798 - val_loss: 0.9397 - val_accuracy: 0.6941
Epoch 5/10
391/391 [==============================] - 69s 175ms/step - loss: 0.9687 - accuracy: 0.6791 - val_loss: 0.8345 - val_accuracy: 0.7248
Epoch 6/10
391/391 [==============================] - 69s 177ms/step - loss: 0.9705 - accuracy: 0.6789 - val_loss: 0.8378 - val_accuracy: 0.7214
Epoch 7/10
391/391 [==============================] - 68s 173ms/step - loss: 0.9735 - accuracy: 0.6793 - val_loss: 0.7720 - val_ac

In [ ]:
def generator():
    i = 0
    while 1:
        i += 1
        yield i
for item in generator():
    print(item)
    if item > 4:
        break